In [2]:
import time
t0 = time.time()

In [3]:
import mysql.connector
import time
import calendar
from datetime import date
from currency_converter import CurrencyConverter
import pandas as pd
import openpyxl
p = CurrencyConverter()

In [4]:
startdate = date(2018,10,1)

In [5]:
str(startdate)

'2018-10-01'

In [6]:
timestamp = calendar.timegm(startdate.timetuple())
#timestamp = calendar.timegm(startoftheMonth.timetuple())

In [7]:
wb = openpyxl.load_workbook('Tradeview activity AUGUST 17 (3).xlsx')
daily = wb['OCT18']

df = pd.DataFrame(daily.values)
df = df[[0,1,2,3,4,5,6]]
df.columns = ['TICKET','LOGIN','DATE','AmountInUSD','COMMENT','BROKER','DATABASE']

## MT4

In [8]:
#connect with mysql mt5 database
mydb = mysql.connector.connect(
    host="185.97.161.95",
    user='mt4_user',
    password="W2bp1j22012",
    database='mt4'
)

In [9]:
cur = mydb.cursor()
query = (
  "SELECT LOGIN, COUNTRY, STATUS, BALANCE, CURRENCY FROM mt4_users WHERE (login BETWEEN 300000 AND 399999) OR(login BETWEEN 700000 AND 799999) OR(login BETWEEN 500000 AND 599999) ")
cur.execute(query)
accounts = pd.DataFrame(cur.fetchall(),columns = ['LOGIN', 'COUNTRY', 'STATUS', 'BALANCE', 'CURRENCY'])

In [10]:
accounts.head()

,LOGIN,COUNTRY,STATUS,BALANCE,CURRENCY
0,310001,Colombia,MM,0.0,USD
1,310002,Cyprus,TH,0.0,USD
2,310003,U.A.E.,TH,0.0,USD
3,310004,Lebanon,BS,0.0,USD
4,310005,Slovenia,BS,0.0,USD


In [13]:
enddate = date(2018,10,7)

In [14]:
cur = mydb.cursor()
query = (
  "SELECT TICKET, LOGIN,TIMESTAMP, PROFIT,COMMENT FROM mt4_trades WHERE timestamp > "+str(timestamp)+" AND ((login BETWEEN 300000 AND 399999) OR(login BETWEEN 700000 AND 799999)OR(login BETWEEN 700000 AND 799999) OR(login BETWEEN 500000 AND 599999)) AND CMD = 6")
cur.execute(query)
deals = pd.DataFrame(cur.fetchall(),columns = [ 'TICKET', 'LOGIN','TIMESTAMP' ,'PROFIT','COMMENT'])

In [15]:
deals.head()

,TICKET,LOGIN,TIMESTAMP,PROFIT,COMMENT
0,14494173,704977,1538359758,110000.00,Deposit-CC-MBW-ID:373038-JPY
1,14494182,321169,1538359992,-2500.00,Withdrawal-Uphold-USD
2,14494184,705036,1538360029,113787.00,Deposit-HLP-1538174679-USDJPY
3,14494228,704732,1538360187,200000.00,Deposit-CC-MBW-ID:373065-JPY
4,14494233,322991,1538360324,-275.11,Withdrawal-Uphold-USD


In [16]:
accounts = accounts.set_index('LOGIN')
accountsmap = accounts.to_dict()

In [17]:
DailySales = pd.DataFrame(deals)

DailySales['BROKER']=DailySales['LOGIN'].map(accountsmap['STATUS'])
DailySales['CURRENCY']=DailySales['LOGIN'].map(accountsmap['CURRENCY'])
DailySales['AmountInUSD']=DailySales.apply(lambda x: p.convert(x['PROFIT'] ,x['CURRENCY'], "USD"),axis=1)
DailySales['DATE']=DailySales.apply(lambda x:time.strftime("%Y.%m.%d %H:%M:%S %Z", time.gmtime(x['TIMESTAMP']))
,axis=1)
#DailySales = DailySales[DailySales.apply(lambda x: 'ADJ' not in x['COMMENT'] and 'Adj' not in x['COMMENT'] and 'adj' not in x['COMMENT'] and 'ADj' not in x['COMMENT'] and 'addj' not in x['COMMENT'] and 'AJ' not in x['COMMENT'] and 'MAM'not in x['COMMENT'],axis=1)]
DailySales = DailySales[DailySales.apply(lambda x: 'Deposit' in x['COMMENT'] or 'Withdraw' in x['COMMENT'] or 'AUTO' in x['COMMENT'],axis=1)]
DailySales = DailySales.reset_index()

#DailySales.drop(['index'],axis=1)
df4 = DailySales[['TICKET','LOGIN','DATE','AmountInUSD','COMMENT','BROKER']]

#print(result.head())

In [18]:
df4['DATABASE'] = 'MT4'

C:\Users\Intern\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
df4.head()

,TICKET,LOGIN,DATE,AmountInUSD,COMMENT,BROKER,DATABASE
0,14494173,704977,2018.10.01 02:09:18 Eastern Standard Time,990.238982,Deposit-CC-MBW-ID:373038-JPY,KY,MT4
1,14494182,321169,2018.10.01 02:13:12 Eastern Standard Time,-2500.000000,Withdrawal-Uphold-USD,AS,MT4
2,14494184,705036,2018.10.01 02:13:49 Eastern Standard Time,1024.330209,Deposit-HLP-1538174679-USDJPY,KY,MT4
3,14494228,704732,2018.10.01 02:16:27 Eastern Standard Time,1800.434513,Deposit-CC-MBW-ID:373065-JPY,KY,MT4
4,14494233,322991,2018.10.01 02:18:44 Eastern Standard Time,-275.110000,Withdrawal-Uphold-USD,AS,MT4


## MT5

In [20]:
#connect with mysql mt5 database
mydb5 = mysql.connector.connect(
    host='109.75.172.115',
    user='tradeview',
    password='tradeview123',
    database='mt5'
)

In [21]:
cur = mydb5.cursor()
query = (
  "SELECT LOGIN,`group`, STATUS, BALANCE FROM mt5_users WHERE status IS NOT NULL")
cur.execute(query)
accounts5 = pd.DataFrame(cur.fetchall(),columns = ['LOGIN', 'group', 'STATUS', 'BALANCE'])

In [22]:
cur = mydb5.cursor()
query = (
  "SELECT deal, LOGIN,`time`, PROFIT,COMMENT FROM mt5_deals WHERE `time` > '"+str(startdate)+"' AND action = 2")
cur.execute(query)
deals5 = pd.DataFrame(cur.fetchall(),columns = [ 'TICKET', 'LOGIN','DATE' ,'PROFIT','COMMENT'])

In [23]:
len(deals5)

0

In [24]:
#mt5_group currency map

In [25]:
cur = mydb5.cursor()
query = (
  "SELECT `group`, currency FROM mt5_groups")
cur.execute(query)
group5 = pd.DataFrame(cur.fetchall(),columns = [ 'group','currency'])

In [26]:
group5 = group5.set_index('group')
groupmap = group5.to_dict()

In [27]:
accounts5['currency']=accounts5['group'].map(groupmap['currency']) 
accounts5 = accounts5.set_index('LOGIN')
accountsmap5 = accounts5.to_dict()

In [28]:
deals5['BROKER']=deals5['LOGIN'].map(accountsmap5['STATUS'])
deals5['CURRENCY']=deals5['LOGIN'].map(accountsmap5['currency'])
deals5 = deals5.dropna(axis=0)
deals5['AmountInUSD']=deals5.apply(lambda x: p.convert(x['PROFIT'] ,x['CURRENCY'], "USD"),axis=1)
deals5 = deals5[deals5.apply(lambda x: 'Deposit' in x['COMMENT'] or 'Withdraw' in x['COMMENT'] or 'AUTO' in x['COMMENT'],axis=1)]
deals5 = deals5.reset_index()

df5 = deals5[['TICKET','LOGIN','DATE','AmountInUSD','COMMENT','BROKER']]

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
df5['DATABASE'] = 'MT5'

In [ ]:
frames = [df, df4,df5]

In [29]:
frames = [df, df4]

result = pd.concat(frames)
result = result.set_index('TICKET')
result = result.drop(['DEAL'])
result = result[~result.index.duplicated(keep='last')]
result = result[result['BROKER']!='']
result = result.dropna()
result = result.reset_index()

In [30]:
result.head()

,TICKET,LOGIN,DATE,AmountInUSD,COMMENT,BROKER,DATABASE
0,14494173,704977,2018.10.01 02:09:18 Eastern Standard Time,990.239,Deposit-CC-MBW-ID:373038-JPY,KY,MT4
1,14494182,321169,2018.10.01 02:13:12 Eastern Standard Time,-2500,Withdrawal-Uphold-USD,AS,MT4
2,14494184,705036,2018.10.01 02:13:49 Eastern Standard Time,1024.33,Deposit-HLP-1538174679-USDJPY,KY,MT4
3,14494228,704732,2018.10.01 02:16:27 Eastern Standard Time,1800.43,Deposit-CC-MBW-ID:373065-JPY,KY,MT4
4,14494233,322991,2018.10.01 02:18:44 Eastern Standard Time,-275.11,Withdrawal-Uphold-USD,AS,MT4


In [31]:
for x in result.index:
    daily.cell(row=x+2, column=1, value=result['TICKET'][x])
    daily.cell(row=x+2, column=2, value=result['LOGIN'][x])
    daily.cell(row=x+2, column=3, value=result['DATE'][x])
    daily.cell(row=x+2, column=4, value=result['AmountInUSD'][x])
    daily.cell(row=x+2, column=5, value=result['COMMENT'][x])
    #print(result['COMMENT'][x])
    daily.cell(row=x+2, column=6, value=result['BROKER'][x])
    daily.cell(row=x+2, column=7, value=result['DATABASE'][x])


In [32]:
#get brokers information from the excel sheet
brokers = pd.read_excel('#MasterList#.xlsx')
all_ini = set(brokers['Status'])
all_brokers = set(brokers['Tradeview Broker List'])
brokers = brokers.set_index('Status')
brokers_dict = brokers.to_dict()

In [33]:
brokers_dict['Tradeview Broker List']['BS']

'Ben Sadgrove'

In [34]:
report = {}

In [35]:
related_status = {x: [] for x in all_brokers}
shared = {x:0 for x in set(result['BROKER'])}

In [36]:
for i in set(result['BROKER']):
    if len(i) <=3:
        name = brokers_dict['Tradeview Broker List'][i]
        related_status[name] = [i]
        shared[i] = 1
    elif len(i) == 4:
        name1 = brokers_dict['Tradeview Broker List'][i[:2]]
        name2 = brokers_dict['Tradeview Broker List'][i[2:]]
        related_status[name1].append(i)
        related_status[name2].append(i)
        shared[i] = 2
    elif len(i) == 5:
        if i[:3] in all_ini:
            name1 = brokers_dict['Tradeview Broker List'][i[:3]]
            name2 = brokers_dict['Tradeview Broker List'][i[3:]]
            related_status[name1].append(i)
            related_status[name2].append(i)
        else:
            name1 = brokers_dict['Tradeview Broker List'][i[:2]]
            related_status[name1].append(i)
            name2 = brokers_dict['Tradeview Broker List'][i[2:]]
            related_status[name2].append(i)
        shared[i] = 2
    elif len(i) ==6:
        if i[:3] in all_ini and i[3:] in all_ini:
            name1 = brokers_dict['Tradeview Broker List'][i[:3]]
            related_status[name1].append(i)
            name2 = brokers_dict['Tradeview Broker List'][i[3:]]
            related_status[name2].append(i)
            shared[i] = 2
        else:
            name1 = brokers_dict['Tradeview Broker List'][i[:2]]
            name2 = brokers_dict['Tradeview Broker List'][i[2:4]]
            name3 = brokers_dict['Tradeview Broker List'][i[4:]]
            related_status[name1].append(i)
            related_status[name2].append(i)
            related_status[name3].append(i)
            shared[i] = 3

In [37]:
result['shared'] = result['BROKER'].map(shared)
result['amount'] = result['AmountInUSD']/result['shared']

In [38]:
result

,TICKET,LOGIN,DATE,AmountInUSD,COMMENT,BROKER,DATABASE,shared,amount
0,14494173,704977,2018.10.01 02:09:18 Eastern Standard Time,990.239,Deposit-CC-MBW-ID:373038-JPY,KY,MT4,1,990.239
1,14494182,321169,2018.10.01 02:13:12 Eastern Standard Time,-2500,Withdrawal-Uphold-USD,AS,MT4,1,-2500
2,14494184,705036,2018.10.01 02:13:49 Eastern Standard Time,1024.33,Deposit-HLP-1538174679-USDJPY,KY,MT4,1,1024.33
3,14494228,704732,2018.10.01 02:16:27 Eastern Standard Time,1800.43,Deposit-CC-MBW-ID:373065-JPY,KY,MT4,1,1800.43
4,14494233,322991,2018.10.01 02:18:44 Eastern Standard Time,-275.11,Withdrawal-Uphold-USD,AS,MT4,1,-275.11
5,14494242,323409,2018.10.01 02:20:05 Eastern Standard Time,500,Deposit-HLP-1538188295-USD,BS,MT4,1,500
6,14494281,704843,2018.10.01 02:23:38 Eastern Standard Time,2718.58,Deposit-HLP-1538196508-USDJPY,KY,MT4,1,2718.58
7,14494311,705027,2018.10.01 02:27:00 Eastern Standard Time,1024.62,Deposit-HLP-1538207874-USDJPY,AL,MT4,1,1024.62
8,14494719,323439,2018.10.01 02:30:53 Eastern Standard Time,250,Deposit-HLP-1538210618-USD,RB,MT4,1,250
9,14494738,321806,2018.10.01 02:33:01 Eastern Standard Time,-200,Withdrawal-Uphold-USD,AS,MT4,1,-200


In [39]:
related_status

{'Alejandro Sanchez': ['ALS'],
 'Mark Lee': ['ML'],
 'Raimondo Perfetto': ['RP'],
 'Maria Isaza': ['MI', 'MIMT'],
 'Mona Gad': ['MG'],
 'Jesus Gomez': ['JG'],
 'Ricardo Grados': ['RG'],
 'Mike Lombardo ': ['MLM', 'BOSMLM'],
 'Sherry Eddy': ['ALSE'],
 'Ryan Boltman': ['RB'],
 'Kazem Ghouchani': ['KG'],
 'Salvador Palma': [],
 'Kengo Yamada': ['KY'],
 'Luis Pinto': ['LP'],
 'Ashutosh Gedamkar': ['ASH'],
 'Jin Yau': [],
 'Diego Victorino': ['DV'],
 'Woo Chung': ['WC'],
 'Andres Salazar': ['AS'],
 'Rosario Bue': ['ROB'],
 'House': ['MLTV'],
 'Adam Saward': ['ADS'],
 'Eliman Dambell': ['ED'],
 'Mike Turck': ['MT', 'MIMT'],
 'Ben Sadgrove': ['BS'],
 'Jose Jaen': [],
 'Andrea Lefleche': ['AL', 'ALSE'],
 'Demar Ruiz': ['DR'],
 'Jose Pino': ['JP', 'ASJP'],
 'Borja Salvador': ['BOS'],
 'Yukiko Sekiai': []}

In [41]:
brokers = pd.read_excel('#MasterList#.xlsx')


In [42]:
for name in all_brokers:
    print(name)
    related_deals_status = related_status[name]
    print(related_deals_status)
    thebroker = result.set_index('BROKER').loc[related_deals_status]
    #print(thebroker)
    deposit = thebroker[thebroker['amount'] >= 0].sum()['amount']
    withdrawal = thebroker[thebroker['amount'] < 0].sum()['amount']
    net = deposit + withdrawal
    report[name] = [deposit,withdrawal,net]

Alejandro Sanchez
['ALS']
Mark Lee
['ML']
Raimondo Perfetto
['RP']
Maria Isaza
['MI', 'MIMT']
Mona Gad
['MG']
Jesus Gomez
['JG']
Ricardo Grados
['RG']
Mike Lombardo 
['MLM', 'BOSMLM']
Sherry Eddy
['ALSE']
Ryan Boltman
['RB']
Kazem Ghouchani
['KG']
Salvador Palma
[]
Kengo Yamada
['KY']
Luis Pinto
['LP']
Ashutosh Gedamkar
['ASH']
Jin Yau
[]
Diego Victorino
['DV']
Woo Chung
['WC']
Andres Salazar
['AS']
Rosario Bue
['ROB']
House
['MLTV']
Adam Saward
['ADS']
Eliman Dambell
['ED']
Mike Turck
['MT', 'MIMT']
Ben Sadgrove
['BS']
Jose Jaen
[]
Andrea Lefleche
['AL', 'ALSE']
Demar Ruiz
['DR']
Jose Pino
['JP', 'ASJP']
Borja Salvador
['BOS']
Yukiko Sekiai
[]


In [43]:
report

{'Alejandro Sanchez': [15784.34, 0.0, 15784.34],
 'Mark Lee': [23739.7, -21439.7, 2300.0],
 'Raimondo Perfetto': [13922.399999999998, 0.0, 13922.399999999998],
 'Maria Isaza': [9066.175, -4860.17, 4206.004999999999],
 'Mona Gad': [21455.189999999995, -2378.41, 19076.779999999995],
 'Jesus Gomez': [1900.9610838291676, -217.252695294762, 1683.7083885344057],
 'Ricardo Grados': [832.2695614809516, -7655.0, -6822.730438519048],
 'Mike Lombardo ': [188790.91999999998, -4767.0, 184023.91999999998],
 'Sherry Eddy': [129.0, -22.5, 106.5],
 'Ryan Boltman': [3062.559122961903, 0.0, 3062.559122961903],
 'Kazem Ghouchani': [7441.522799999999,
  -26325.04596739045,
  -18883.523167390453],
 'Salvador Palma': [0.0, 0.0, 0.0],
 'Kengo Yamada': [294080.19271648035, -277389.7807278088, 16690.411988671578],
 'Luis Pinto': [0.0, -1800.0, -1800.0],
 'Ashutosh Gedamkar': [439.0, -982.0, -543.0],
 'Jin Yau': [0.0, 0.0, 0.0],
 'Diego Victorino': [5920.5, -1359.0, 4561.5],
 'Woo Chung': [33285.0, -10951.66, 22

In [ ]:
ini = 'KY'

In [ ]:
name = 'Kengo Yamada'
related_deals_status = list(filter(lambda x: ini in x, set(result['BROKER'])))
thebroker = result.set_index('BROKER').loc[related_deals_status]
deposit = thebroker[thebroker['amount'] >= 0].sum()['amount']
withdrawal = thebroker[thebroker['amount'] < 0].sum()['amount']
net = deposit + withdrawal
report[name] = [deposit,withdrawal,net]

In [ ]:
thebroker.sum()['amount']

In [ ]:
net

In [45]:
broker_report = pd.DataFrame(report).T
broker_report = broker_report.rename(columns={0: "Deposit", 1: "Withdrawal",2:"Net"})
broker_report = broker_report.sort_values(by=['Deposit'],ascending=False)

In [46]:
for i in range(len(broker_report)):
    daily.cell(row=13+i, column=11, value=broker_report.iloc[i].name)
    daily.cell(row=13+i, column=12, value=broker_report.iloc[i].Deposit)
    daily.cell(row=13+i, column=13, value=broker_report.iloc[i].Withdrawal)
    daily.cell(row=13+i, column=14, value=broker_report.iloc[i].Net)

In [47]:
wb.save(filename='Tradeview activity AUGUST 17 (3).xlsx')

In [48]:
t1 = time.time()

print('The daily sales report is finished, it takes ' +str(int(t1-t0))+' seconds' )

The daily sales report is finished, it takes 201 seconds
